In [1]:
import numpy as np

import scanpy as sc
import scvelo as scv
scv.settings.verbosity = 0

import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('/data/nelkazwi/code/cisTopic/')
from cisTopic import cisTopic


In [3]:
path = '/data/nelkazwi/code/UniTVelo-ATAC/notebooks/data/HSPC_dataset/'
adata_rna = sc.read(path+"adata_rna_prepro.h5ad")
adata_atac = sc.read(path+"adata_atac_prepro.h5ad")


In [6]:
min_regions = 800
sc.pp.filter_genes(adata_atac, min_counts=min_regions)
sc.pp.filter_cells(adata_atac, min_counts=1)
adata_atac

AnnData object with n_obs × n_vars = 11605 × 93383
    obs: 'celltype', 'n_counts'
    var: 'chrom', 'chromStart', 'chromEnd', 'n_counts'

In [5]:
n_samples = 3000
n_burnin = 10
T = 30
n_b_samples = int(n_samples/n_burnin)
batch_size = 500
alpha = 50/T
beta = 0.1


cis_t = cisTopic(adata_atac, T, alpha, beta)

In [ ]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
print(f'Running on: {device}')

lmbda, psi = cis_t.fit(batch_size, n_samples, n_burnin, dev=device, save_data=False)

In [12]:

path_samples = '/scratch/nelkazwi/RNA_velo/Unitvelo_atac/HSPC/30_topics/'

lmbda=torch.load(path_samples+"theta_ATAC_"+str(T)+".txt",map_location=torch.device('cpu'))
m=lmbda[50:,:,:].mean(axis=0)
m=m/m.sum(axis=1)[:,np.newaxis] 

psi = torch.load(path_samples+"phi_ATAC_"+str(T)+".txt",map_location=torch.device('cpu'))
m_psi = psi[50:,:,:].mean(axis=0)
m_psi =m_psi /m_psi.sum(axis=1)[:,np.newaxis]

In [16]:
phi = torch.matmul(m,m_psi)
adata_atac.obsm["cisTopic"] = phi.numpy()
adata_atac.write_h5ad("UniTVelo-ATAC/notebooks/data/HSPC_dataset/adata_atac_postpro.h5ad")

In [19]:
adata_atac.var

,chrom,chromStart,chromEnd,n_counts
chr1:181070-181803,chr1,181070,181803,1721.0
chr1:191121-192037,chr1,191121,192037,813.0
chr1:267568-268459,chr1,267568,268459,867.0
chr1:629500-630394,chr1,629500,630394,2781.0
chr1:633580-634477,chr1,633580,634477,14154.0
...,...,...,...,...
KI270721.1:2095-2980,KI270721.1,2095,2980,833.0
KI270726.1:41475-42286,KI270726.1,41475,42286,1095.0
KI270713.1:21418-22293,KI270713.1,21418,22293,13315.0
KI270713.1:29583-30478,KI270713.1,29583,30478,3828.0


In [21]:
adata_rna.var

,Accession,chrom,chromEnd,chromStart,Strand
Gene,,,,,
BX004987.1,ENSG00000278704,chrGL000009.2,58376,56140,-
AC145212.1,ENSG00000277400,chrGL000194.1,115018,53590,-
MAFIP,ENSG00000274847,chrGL000194.1,115055,53594,-
AC011043.1,ENSG00000276256,chrGL000195.1,49164,42939,-
AC011043.2,ENSG00000278198,chrGL000195.1,179372,173872,+
...,...,...,...,...,...
TTTY4B,ENSG00000235412,chrY,24607025,24570202,+
BPY2B,ENSG00000183795,chrY,24639207,24607560,+
DAZ4,ENSG00000205916,chrY,24907040,24833843,+
